In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));
import pandas as pd
import numpy as np

<font size=18>Homework 11: Recommender Systems 2</font>

# Build a Baseline Model and Compute the RMSE

The file **rating_final.csv** (found in the presentation download for this lesson) contains user ratings for overall, food, and service for various restaurants.   

Do the following:

* display the first 5 lines of the data (get familiar with the data frame)
* find the minimum restaurant rating
* find the maximum restaurant rating
* adjust the rating scale by shifting up 1 if 0 is included
* calculate the mean restaurant rating for all restaurant (just to get an idea)
* drop the ratings for food and service so that only the overall rating remains
* split the data set so that 80\% of a users ratings are in the training set and 20\% are in the testing set
* build a baseline model that assigns the appropriate rating for all predictions and compute the RMSE of these on the testing set

Click <a href = https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings> here </a> or <a href = https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data> here </a> for more details about the data set.

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [2]:
df = pd.read_csv("data/restaurant/rating_final.csv")
df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [6]:
print(df.rating.min(), df.rating.max())

if df.rating.isin([0]).any():
    df['rating'] = df['rating'] + 1

1 3


In [7]:
round(df['rating'].mean(), 2)

2.2

In [8]:
df = df.drop(['food_rating','service_rating'],axis=1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

x = df.copy()
y = df['userID']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, stratify=y, random_state=42)

In [10]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline(user_id, place_id):
    return 2

def score(cf_model, x_test):
    id_pairs = zip(x_test['userID'], x_test['placeID'])
    y_pred = np.array([cf_model(user, place) for (user, place) in id_pairs])
    y_true = np.array(x_test['rating'])
    return rmse(y_true, y_pred)

In [11]:
score(baseline, x_test)

0.7996780468043202

# Build a Weighted Mean User-Based Filter

From data in the file **rating_final.csv**, build a ratings matrix from the data frame of users, places, and restaurant ratings and build a user-based collaborative filtering model that weights mean rank using cosine similarity among users.  Fit the model on the training set and compute the RMSE for this model using the test set and compare it to the RMSE of the baseline model.  Is it better than baseline?  (*i.e.* is the RMSE smaller?)

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [12]:
x_train.head()

,userID,placeID,rating
830,U1089,132875,2
229,U1014,135043,2
256,U1134,135046,3
1039,U1073,132951,1
56,U1021,132740,3


In [13]:
r_matrix = x_train.pivot(values='rating', index='userID', columns='placeID')
r_matrix.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Import cosine_score 
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity matrix using the dummy ratings matrix
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

In [ ]:
#Convert into pandas dataframe 
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head()

In [ ]:
#User Based Collaborative Filter using Weighted Mean Ratings
def cf_user_wmean(user_id, book_id):
    
    #Check if book_id exists in r_matrix and if there is overlap with other 
    # users, 
#    sim_scores = cosine_sim[user_id]
#    sss = sum(sim_scores)
    if book_id in r_matrix:
        
        #Get the similarity scores for the user in question with every other user
        sim_scores = cosine_sim[user_id]
        
        #Get the user ratings for the movie in question
        m_ratings = r_matrix[book_id]
        
        #Extract the indices containing NaN in the m_ratings series
        idx = m_ratings[m_ratings.isnull()].index
                
        #Drop the NaN values from the m_ratings Series
        m_ratings = m_ratings.dropna()
        
        #Drop the corresponding cosine scores from the sim_scores series
        sim_scores = sim_scores.drop(idx)
        
        #Compute the final weighted mean
        if sim_scores.sum()>0:
            wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
        else:  # user had zero cosine similarity with other users
            wmean_rating = 6.0
    
    else:
        #Default to a rating of 6.0 in the absence of any information
        wmean_rating = 6.0
    
    return wmean_rating

In [ ]:
score(cf_user_wmean)

# Build a kNN-Based Collaborative Filter

From data in the file **rating_final.csv**, use the *surprise* library in Python to build an kNN-based collaborative filtering model for the restaurant ratings.  Fit the model on the training set and compute the RMSE for this model on the test and compare it to the RMSEs of the baseline and weighted mean user-based models.

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [ ]:
# enter your code here

# Build a Hybrid Filter

From data in the files **rating_final.csv** and **geoplaces2.csv** build a recommender system that is a hybrid of a metadata content-based recommender and the SVD collaborative filter.  Your recommender should do the following:

* Take in a user ID and restaurant name as user input
* Use a metadata content-based model to compute the 25 most similar restaurants based on cosine similarity from the following details (create a soup as done for the content-based recommender from Lesson 10)
    - price
    - dress code
    - accessibility
    - ambiance
    - alcohol
    - smoking area
* Compute the predicted ratings that the user might give to these 25 restaurants using the SVD collaborative filter
* Print price, dress code, accessibility, ambiance, alcohol, and smoking area to see if they are similar for the predicted restaurants.
* Return the top 10 restaurant recommendations along with their predicted ratings when user **U1077** enters the restaurant named **Restaurante Tiberius**. 
* Also return the top 10 restaurant recommendations along with their predicted ratings when user **U1065** enters the restaurant named **Restaurante Tiberius** and comment on the similarities and differences in the resulting recommendations. 

*Note 1: This data set does not have the issue of having two different ID's in separate files for each restaurant like the movie data used in the textbook example, so you won't need to use the cell for mapping ID's to titles.*

*Note 2: because of the small number of words in the "soup", many of the restaurant pairs have a cosine similarity of 1, which will affect the predicted ratings and recommendations.  This means you will have to use a different method for excluding the cosine similarity of the item with itself (use the **del** function).* 

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [ ]:
# enter your code here